In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import cPickle as pickle
import collections
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN
from video_process_utils import *
import statsmodels.api as sm

Using TensorFlow backend.
/home/bryan/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
target_col = 'SEMLS_dev_residual'

In [3]:
alldata_processed =\
    pd.read_csv("data/annotations/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed['target_count'] = alldata_processed.groupby('videoid')[target_col].transform(lambda x: x.count())

In [4]:
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed[alldata_processed['target_count'] == 2]
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [5]:
datasplit_df = pd.read_csv('./data/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [6]:
with open('./data/all_processed_video_segments.pickle', 'r') as handle:
    processed_video_segments = pickle.load(handle)

In [7]:
assert(np.sum(alldata_processed[target_col].isnull()) == 0)

In [9]:
x_columns_left = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,50,52,54,56]
x_columns_right = [2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,51,53,55,57]

target_dict_L = {}
target_dict_R = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    if row['side'] == 'L':
        target_dict_L[row['videoid']] = row[target_col]
    if row['side'] == 'R':
        target_dict_R[row['videoid']] = row[target_col]

In [10]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)
X = np.concatenate([X[:,:,x_columns_left],X[:,:,x_columns_right]])

In [11]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)
X_train = np.concatenate([X_train[:,:,x_columns_left],X_train[:,:,x_columns_right]])

In [12]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)
X_validation = np.concatenate([X_validation[:,:,x_columns_left],X_validation[:,:,x_columns_right]])

In [13]:
y_train_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train = np.array(y_train_L + y_train_R)

In [14]:
y_validation_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation = np.array(y_validation_L + y_validation_R)

In [15]:
y_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in all_ids]
y_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in all_ids]
y = np.array(y_L + y_R)

In [16]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [17]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = train_videoid_weights + train_videoid_weights
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = validation_videoid_weights + validation_videoid_weights
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [18]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)

In [19]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)
y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
#c_i_factor is just a constant initially introduced to make the loss function more interpretable, but
#is probably not necessary if the model were to be retrained
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights])) 

In [20]:
vid_length = 124

In [29]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#this is just for monitoring
mse_metric = w_mse(target_range**2*np.array(weights))

K.clear_session()

model = Sequential()
model.add(Conv1D(conv_dim,filter_length, input_dim=X_train.shape[2],input_length=vid_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='linear'))

/home/bryan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 8, padding="same", input_shape=(124, 12))`


In [30]:
checkpoint_folder = "/home/bryan/clinical-video/cnn_checkpoints_%s" % (target_col)

In [ ]:
train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss_2:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)


    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_2', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=32, epochs=n_epochs,shuffle=True)

In [31]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [32]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename'].apply(lambda x: int(x.split('-')[1]))
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [33]:
#todo: change so we also have a X,y for all_ids
#then for each of 100 epochs, generate the predictions of the model for each epoch
#this dataframe can then be saved, and a file called "get_best_epoch" can be used to generate the best one
#(video_id,target,predictions,dataset)

In [34]:
def predict_and_aggregate(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    side_col = ['L' if i < len(df)/2 else 'R' for i in range(len(df))]
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    videoids = videoids + videoids
    df["videoid"] = np.array(videoids)
    df['side'] = side_col
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid','side'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [35]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
video_ids = video_ids + video_ids
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
side_col = ['L' if i < len(predictions_df)/2 else 'R' for i in range(len(predictions_df))]
predictions_df[target_col] = y
predictions_df['side'] = side_col
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [36]:
for i in range(0,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

weights-00-86.4237.hdf5
weights-01-98.9111.hdf5
weights-02-87.3220.hdf5
weights-03-85.8565.hdf5
weights-04-91.1653.hdf5
weights-05-80.9056.hdf5
weights-06-72.2637.hdf5
weights-07-59.8240.hdf5
weights-08-67.0776.hdf5
weights-09-60.6945.hdf5
weights-10-70.8957.hdf5
weights-11-62.5470.hdf5
weights-12-65.3806.hdf5
weights-13-66.4281.hdf5
weights-14-73.1030.hdf5
weights-15-60.8615.hdf5
weights-16-64.0410.hdf5
weights-17-62.3888.hdf5
weights-18-60.2654.hdf5
weights-19-72.5628.hdf5
weights-20-62.0319.hdf5
weights-21-63.2341.hdf5
weights-22-72.9835.hdf5
weights-23-64.8781.hdf5
weights-24-64.4222.hdf5
weights-25-72.8902.hdf5
weights-26-60.4510.hdf5
weights-27-66.1328.hdf5
weights-28-58.3316.hdf5
weights-29-64.2199.hdf5
weights-30-63.4656.hdf5
weights-31-62.6747.hdf5
weights-32-59.5008.hdf5
weights-33-62.0990.hdf5
weights-34-64.9120.hdf5
weights-35-62.6186.hdf5
weights-36-59.0109.hdf5
weights-37-62.5512.hdf5
weights-38-59.0609.hdf5
weights-39-61.5603.hdf5
weights-40-69.5490.hdf5
weights-41-56.34

In [ ]:
predictions_df.groupby(['videoid','side','dataset'],as_index=False).mean().to_csv("./predictions/cnn_%s_predictions_all_epochs.csv" % (target_col),index=False)